# Gender Detection using Gaussian Mixture Models

## Setup
We first need to import the required Python libraries for the entire anaylsis  
Prerequisites required in this implementation are:
```
Python 3.x 
numpy   
scipy
matplotlib
scikit-learn
librosa
```

You can invoke any required library into your workspace as:  
`import package_name`  

In [ ]:
import os
import pickle
import numpy as np
from scipy.io.wavfile import read
from sklearn.mixture import GaussianMixture as GMM
from sklearn import preprocessing
import librosa
import warnings

#Ignore warnings for now
warnings.filterwarnings("ignore")

### Load training data

In [ ]:
training_path = '../pygender/train_data'

male_files = []
female_files = []

# r=root, d=directories, f = files
for r, d, f in os.walk(os.path.join(training_path,'male')):
    for file in f:
        if '.wav' in file:
            male_files.append(os.path.join(r, file))
for r, d, f in os.walk(os.path.join(training_path,'female')):
    for file in f:
        if '.wav' in file:
            female_files.append(os.path.join(r, file))
print('Number of male files = ', len(male_files))
print('Number of female files = ', len(female_files))

### Feature extraction
Now let's extract MFCC features that we will use for training our GMMs.

We will first train a GMM model for female speech.

In [ ]:
def get_MFCC(sampling_rate,audio):
    features = librosa.feature.mfcc(audio,sampling_rate, win_length = int(0.025*sampling_rate), hop_length = int(0.01*sampling_rate), n_mfcc = 13, htk=True)
    features = np.transpose(features)
    features = preprocessing.scale(features)
    return features

In [ ]:
features = np.asarray(())
for f in female_files:
    sampling_rate,audio = read(f)
    vector   = get_MFCC(sampling_rate,audio.astype(float))
    if features.size == 0:
        features = vector
    else:
        features = np.vstack((features, vector))
print('Feature vector shape: ',features.shape)

### Fitting a GMM


In [ ]:
female_gmm = GMM(n_components = 8, max_iter = 200, covariance_type='diag',n_init = 3)
female_gmm.fit(features)

### Saving the trained model
We use the _pickle_ module to save ("pickle") the trainel GMM model, which can be used whie testing.

In [ ]:
picklefile = "models/female.gmm"
pickle.dump(female_gmm,open(picklefile,'wb'))

Similarly, we train a Gaussian Mixture Model to model male speech

In [ ]:
features = np.asarray(());
for f in male_files:
    sampling_rate,audio = read(f)
    vector   = get_MFCC(sampling_rate,audio.astype(float))
    if features.size == 0:
        features = vector
    else:
        features = np.vstack((features, vector))

male_gmm = GMM(n_components = 8, max_iter = 200, covariance_type='diag',n_init = 3)
male_gmm.fit(features)

picklefile = "models/male.gmm"
pickle.dump(male_gmm,open(picklefile,'wb'))

# Test

Let's load the test files, and the pickled models

In [ ]:
test_path = '../pygender/test_data'
model_path = 'models'
gmm_files = [os.path.join(model_path,fname) for fname in os.listdir(model_path) if fname.endswith('.gmm')]
print(gmm_files)

Unpickling to retrieve the saved models

In [ ]:
models = [pickle.load(open(fname,'rb')) for fname in gmm_files]

In [ ]:
files = [[], []]

true = [0,0]
false = [0,0]
total = [0,0]
for j, gen in enumerate(['female', 'male']):
    # r=root, d=directories, f = files
    for r, d, f in os.walk(os.path.join(test_path, gen)):
        for file in f:
            if '.wav' in file:
                files[j].append(os.path.join(r, file))
    for f in files[j]:
        # print(f.split("/")[-1])
        sampling_rate, audio  = read(f)
        features   = get_MFCC(sampling_rate,audio.astype(float))
        scores     = None
        log_likelihood = np.zeros(len(models))
        for i in range(len(models)):
            gmm    = models[i]         #checking with each model one by one
            scores = np.array(gmm.score(features))
            log_likelihood[i] = scores.sum()
        winner = np.argmax(log_likelihood)
        if(winner == j):
            true[j]+=1
        else:
            false[j]+=1
        total[j]+=1

Calculate and print accuracy

In [ ]:
accuracy = (sum(true)/sum(total))
print("Accuracy = ", accuracy*100, "%")

Now try recording your voice, and test if the GMM works!

In [ ]:
audio_file_path = 'manan.wav'
dict = {0:'FEMALE', 1:'MALE'}
sampling_rate, audio  = read(audio_file_path)
features   = get_MFCC(sampling_rate,audio.astype(float))
scores     = None
log_likelihood = np.zeros(len(models))
for i in range(len(models)):
    gmm    = models[i]         #checking with each model one by one
    scores = np.array(gmm.score(features))
    log_likelihood[i] = scores.sum()
winner = np.argmax(log_likelihood)
print(dict[winner])